# <center><h1>Companhia Aberta Demonstrativo Financeiro</h1></center>

# <h2>Load Libraries</h2>

In [1]:
import pandas as pd
import numpy as np

import sys

# <h2>Import Data</h2>

## <h3>Load .csv</h3>

In [2]:
cia_aberta_df = pd.read_csv(r'dfp_cia_aberta_BPA_con_2020.csv', encoding='ISO-8859-1', sep=";")    # leio o .csv com os dados de todas as companhias abertas

## <h3>Select Company</h3>

In [3]:
df = cia_aberta_df[cia_aberta_df['CNPJ_CIA'] == '97.837.181/0001-47'].copy()    # (seleciono somente as linhas relativas a uma companhia de interesse)
df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']                                         # seleciono somente o último ou penúltimo exercício
df = df[['CD_CONTA','DS_CONTA','VL_CONTA']]
df.reset_index(inplace=True, drop=True)

In [4]:
df

,CD_CONTA,DS_CONTA,VL_CONTA
0,1,Ativo Total,11498520.0
1,1.01,Ativo Circulante,4220022.0
2,1.01.01,Caixa e Equivalentes de Caixa,1728413.0
3,1.01.02,Aplicações Financeiras,0.0
4,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...
71,1.02.04.02.07,Goodwill na aquisição da Caetex Florestal,8767.0
72,1.02.04.02.08,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
73,1.02.04.02.09,Goodwill na aquisição da Massima Revestimentos...,6110.0
74,1.02.04.02.10,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


### <h4>Subset DataFrame (only for testing purposes)</h4>

In [5]:
#df = df.iloc[np.r_[0:7, 24:31],:]

In [6]:
#df

# <h2>Wrangling</h2>

## <h3>Number of Steps</h3>

In [7]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_len = [len(lst) for lst in cd_conta_split]
num_levels = max(cd_conta_len)

In [8]:
num_levels

5

## <h3>ROUND 1</h3>

In [9]:
round=1

In [10]:
df

,CD_CONTA,DS_CONTA,VL_CONTA
0,1,Ativo Total,11498520.0
1,1.01,Ativo Circulante,4220022.0
2,1.01.01,Caixa e Equivalentes de Caixa,1728413.0
3,1.01.02,Aplicações Financeiras,0.0
4,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...
71,1.02.04.02.07,Goodwill na aquisição da Caetex Florestal,8767.0
72,1.02.04.02.08,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
73,1.02.04.02.09,Goodwill na aquisição da Massima Revestimentos...,6110.0
74,1.02.04.02.10,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


### <h4>Join</h4>

In [11]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_joincol = [row[:round] for row in cd_conta_split]        #cd_conta_joincol: column with keys used to join

Select rows with only one element:

In [12]:
cd_conta_len = [len(row) for row in cd_conta_split]
idx_list = [idx for idx, element in enumerate(cd_conta_len) if element == round]        # round = 1
keys = [cd_conta_joincol[idx] for idx in idx_list]
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_list)}

In [13]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA,VL_CONTA
0,1,Ativo Total,11498520.0


Join:

In [14]:
ds_conta_1 = [df.DS_CONTA[key_idx_dict[key]]
              for key in key_idx_dict
              for row in cd_conta_joincol
              if ".".join(row) == key]

In [15]:
ds_conta_1 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [16]:
if pd.Series(ds_conta_1).equals(df.DS_CONTA):
    df.rename(columns={'DS_CONTA': 'DS_CONTA_1'}, inplace=True)
    df.to_csv("output.csv", index=False)
    sys.exit()
else:
    df.insert(round, column='DS_CONTA_1', value=ds_conta_1)

In [17]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,1,Ativo Total,Ativo Total,11498520.0
1,1.01,Ativo Total,Ativo Circulante,4220022.0
2,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
3,1.01.02,Ativo Total,Aplicações Financeiras,0.0
4,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...,...
71,1.02.04.02.07,Ativo Total,Goodwill na aquisição da Caetex Florestal,8767.0
72,1.02.04.02.08,Ativo Total,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
73,1.02.04.02.09,Ativo Total,Goodwill na aquisição da Massima Revestimentos...,6110.0
74,1.02.04.02.10,Ativo Total,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


### <h4>Drop rows</h4>

Select rows with only one element:

In [18]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,1,Ativo Total,Ativo Total,11498520.0


For each key which was used to join, check if there is at least another row in `CD_CONTA` which code starts with the same key (example: `CD_CONTA` '1.01' starting with '1'; `CD_CONTA` '1.01.01' starting with '1.01', etc.)

In [19]:
for idx in idx_list:
    mask = [True]*len(ds_conta_1)
    mask[idx] = False
    if any(row[:round] == cd_conta_joincol[idx] for row in cd_conta_split[mask]):
        df = df.drop(idx)
    else:
        df.loc[idx, 'CD_CONTA'] = df.loc[idx, 'CD_CONTA'] + '.00'

df.reset_index(inplace=True, drop=True)

In [20]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,1.01,Ativo Total,Ativo Circulante,4220022.0
1,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
2,1.01.02,Ativo Total,Aplicações Financeiras,0.0
3,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.01.02.01.01,Ativo Total,Títulos para Negociação,0.0
...,...,...,...,...
70,1.02.04.02.07,Ativo Total,Goodwill na aquisição da Caetex Florestal,8767.0
71,1.02.04.02.08,Ativo Total,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
72,1.02.04.02.09,Ativo Total,Goodwill na aquisição da Massima Revestimentos...,6110.0
73,1.02.04.02.10,Ativo Total,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


## <h3>ROUND 2</h3>

In [21]:
round=2

In [22]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,1.01,Ativo Total,Ativo Circulante,4220022.0
1,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
2,1.01.02,Ativo Total,Aplicações Financeiras,0.0
3,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.01.02.01.01,Ativo Total,Títulos para Negociação,0.0
...,...,...,...,...
70,1.02.04.02.07,Ativo Total,Goodwill na aquisição da Caetex Florestal,8767.0
71,1.02.04.02.08,Ativo Total,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
72,1.02.04.02.09,Ativo Total,Goodwill na aquisição da Massima Revestimentos...,6110.0
73,1.02.04.02.10,Ativo Total,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


### <h4>Join</h4>

In [23]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_joincol = [row[:round] for row in cd_conta_split]

Select rows with only 2 elements:

In [24]:
cd_conta_len = [len(lst) for lst in cd_conta_split]
idx_list = [idx for idx, element in enumerate(cd_conta_len) if element == round]
keys = [cd_conta_joincol[idx] for idx in idx_list]
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_list)}

In [25]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,1.01,Ativo Total,Ativo Circulante,4220022.0
23,1.02,Ativo Total,Ativo Não Circulante,7278498.0


Join:

In [26]:
ds_conta_2 = [df.DS_CONTA[key_idx_dict[key]]
              for key in key_idx_dict
              for row in cd_conta_joincol
              if ".".join(row) == key]

In [27]:
ds_conta_2 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [28]:
if pd.Series(ds_conta_2).equals(df.DS_CONTA):
    df.rename(columns={'DS_CONTA': 'DS_CONTA_2'}, inplace=True)
    df.to_csv("output.csv", index=False)
    sys.exit()
else:
    df.insert(round, column='DS_CONTA_2', value=ds_conta_2)

In [29]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,1.01,Ativo Total,Ativo Circulante,Ativo Circulante,4220022.0
1,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
2,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
3,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.01.02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
...,...,...,...,...,...
70,1.02.04.02.07,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Caetex Florestal,8767.0
71,1.02.04.02.08,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
72,1.02.04.02.09,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Massima Revestimentos...,6110.0
73,1.02.04.02.10,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


### <h4>Drop rows</h4>

Select rows with only 2 elements:

In [30]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,1.01,Ativo Total,Ativo Circulante,Ativo Circulante,4220022.0
23,1.02,Ativo Total,Ativo Não Circulante,Ativo Não Circulante,7278498.0


For each key which was used to join, check if there is at least another row in `CD_CONTA` which code starts with the same key (example: `CD_CONTA` '1.01' starting with '1'; `CD_CONTA` '1.01.01' starting with '1.01', etc.)

In [31]:
for idx in idx_list:
    mask = [True]*len(ds_conta_2)
    mask[idx] = False
    if any(row[:round] == cd_conta_joincol[idx] for row in cd_conta_split[mask]):
        df = df.drop(idx)
    else:
        df.loc[idx, 'CD_CONTA'] = df.loc[idx, 'CD_CONTA'] + '.00'

df.reset_index(inplace=True, drop=True)

In [32]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
2,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
3,1.01.02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
4,1.01.02.01.02,Ativo Total,Ativo Circulante,Títulos Designados a Valor Justo,0.0
...,...,...,...,...,...
68,1.02.04.02.07,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Caetex Florestal,8767.0
69,1.02.04.02.08,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
70,1.02.04.02.09,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Massima Revestimentos...,6110.0
71,1.02.04.02.10,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


## <h3>ROUND 3</h3>

In [33]:
round=3

In [34]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
2,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
3,1.01.02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
4,1.01.02.01.02,Ativo Total,Ativo Circulante,Títulos Designados a Valor Justo,0.0
...,...,...,...,...,...
68,1.02.04.02.07,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Caetex Florestal,8767.0
69,1.02.04.02.08,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
70,1.02.04.02.09,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Massima Revestimentos...,6110.0
71,1.02.04.02.10,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


### <h4>Join</h4>

In [35]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_joincol = [row[:round] for row in cd_conta_split]

Select rows with only 3 elements:

In [36]:
cd_conta_len = [len(lst) for lst in cd_conta_split]
idx_list = [idx for idx, element in enumerate(cd_conta_len) if element == round]
keys = [cd_conta_joincol[idx] for idx in idx_list]
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_list)}

In [37]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
0,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
7,1.01.03,Ativo Total,Ativo Circulante,Contas a Receber,1318743.0
13,1.01.04,Ativo Total,Ativo Circulante,Estoques,924743.0
14,1.01.05,Ativo Total,Ativo Circulante,Ativos Biológicos,0.0
15,1.01.06,Ativo Total,Ativo Circulante,Tributos a Recuperar,176456.0
17,1.01.07,Ativo Total,Ativo Circulante,Despesas Antecipadas,0.0
18,1.01.08,Ativo Total,Ativo Circulante,Outros Ativos Circulantes,71667.0
22,1.02.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,2071636.0
46,1.02.02,Ativo Total,Ativo Não Circulante,Investimentos,963437.0


Join:

In [38]:
ds_conta_3 = [df.DS_CONTA[key_idx_dict[key]]
              for key in key_idx_dict
              for row in cd_conta_joincol
              if ".".join(row) == key]

In [39]:
ds_conta_3 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [40]:
if pd.Series(ds_conta_3).equals(df.DS_CONTA):
    df.rename(columns={'DS_CONTA': 'DS_CONTA_3'}, inplace=True)
    df.to_csv("output.csv", index=False)
    sys.exit()
else:
    df.insert(round, column='DS_CONTA_3', value=ds_conta_3)

In [41]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
0,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras,0.0
2,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
3,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos para Negociação,0.0
4,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos Designados a Valor Justo,0.0
...,...,...,...,...,...,...
68,1.02.04.02.07,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Caetex Florestal,8767.0
69,1.02.04.02.08,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
70,1.02.04.02.09,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Massima Revestimentos...,6110.0
71,1.02.04.02.10,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


### <h4>Drop rows</h4>

Select rows with only 3 elements:

In [42]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
0,1.01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras,0.0
7,1.01.03,Ativo Total,Ativo Circulante,Contas a Receber,Contas a Receber,1318743.0
13,1.01.04,Ativo Total,Ativo Circulante,Estoques,Estoques,924743.0
14,1.01.05,Ativo Total,Ativo Circulante,Ativos Biológicos,Ativos Biológicos,0.0
15,1.01.06,Ativo Total,Ativo Circulante,Tributos a Recuperar,Tributos a Recuperar,176456.0
17,1.01.07,Ativo Total,Ativo Circulante,Despesas Antecipadas,Despesas Antecipadas,0.0
18,1.01.08,Ativo Total,Ativo Circulante,Outros Ativos Circulantes,Outros Ativos Circulantes,71667.0
22,1.02.01,Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Ativo Realizável a Longo Prazo,2071636.0
46,1.02.02,Ativo Total,Ativo Não Circulante,Investimentos,Investimentos,963437.0


For each key which was used to join, check if there is at least another row in `CD_CONTA` which code starts with the same key (example: `CD_CONTA` '1.01' starting with '1'; `CD_CONTA` '1.01.01' starting with '1.01', etc.)

In [43]:
for idx in idx_list:
    mask = [True]*len(ds_conta_3)
    mask[idx] = False
    if any(row[:round] == cd_conta_joincol[idx] for row in cd_conta_split[mask]):
        df = df.drop(idx)
    else:
        df.loc[idx, 'CD_CONTA'] = df.loc[idx, 'CD_CONTA'] + '.00'

df.reset_index(inplace=True, drop=True)

In [44]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
0,1.01.01.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
2,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos para Negociação,0.0
3,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos Designados a Valor Justo,0.0
4,1.01.02.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...,...,...,...
60,1.02.04.02.07,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Caetex Florestal,8767.0
61,1.02.04.02.08,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
62,1.02.04.02.09,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Massima Revestimentos...,6110.0
63,1.02.04.02.10,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


## <h3>ROUND 4</h3>

In [45]:
round=4

In [46]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
0,1.01.01.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
2,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos para Negociação,0.0
3,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Títulos Designados a Valor Justo,0.0
4,1.01.02.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...,...,...,...
60,1.02.04.02.07,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Caetex Florestal,8767.0
61,1.02.04.02.08,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
62,1.02.04.02.09,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Massima Revestimentos...,6110.0
63,1.02.04.02.10,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


### <h4>Join</h4>

In [47]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_joincol = [row[:round] for row in cd_conta_split]

Select rows with only 4 elements:

In [48]:
cd_conta_len = [len(lst) for lst in cd_conta_split]
idx_list = [idx for idx, element in enumerate(cd_conta_len) if element == round]
keys = [cd_conta_joincol[idx] for idx in idx_list]
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_list)}

In [49]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
0,1.01.01.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.01.02.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,1.01.02.03,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
6,1.01.03.01,Ativo Total,Ativo Circulante,Contas a Receber,Clientes,1239315.0
9,1.01.03.02,Ativo Total,Ativo Circulante,Contas a Receber,Outras Contas a Receber,79428.0
11,1.01.04.00,Ativo Total,Ativo Circulante,Estoques,Estoques,924743.0
12,1.01.05.00,Ativo Total,Ativo Circulante,Ativos Biológicos,Ativos Biológicos,0.0
13,1.01.06.01,Ativo Total,Ativo Circulante,Tributos a Recuperar,Tributos Correntes a Recuperar,176456.0
14,1.01.07.00,Ativo Total,Ativo Circulante,Despesas Antecipadas,Despesas Antecipadas,0.0


Join:

In [50]:
ds_conta_4 = [df.DS_CONTA[key_idx_dict[key]]
              for key in key_idx_dict
              for row in cd_conta_joincol
              if ".".join(row) == key]

In [51]:
ds_conta_4 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [52]:
if pd.Series(ds_conta_4).equals(df.DS_CONTA):
    df.rename(columns={'DS_CONTA': 'DS_CONTA_4'}, inplace=True)
    df.to_csv("output.csv", index=False)
    sys.exit()
else:
    df.insert(round, column='DS_CONTA_4', value=ds_conta_4)

In [53]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA_4,DS_CONTA,VL_CONTA
0,1.01.01.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
2,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos para Negociação,0.0
3,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
4,1.01.02.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...,...,...,...,...
60,1.02.04.02.07,Ativo Total,Ativo Não Circulante,Intangível,Goodwill,Goodwill na aquisição da Caetex Florestal,8767.0
61,1.02.04.02.08,Ativo Total,Ativo Não Circulante,Intangível,Goodwill,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
62,1.02.04.02.09,Ativo Total,Ativo Não Circulante,Intangível,Goodwill,Goodwill na aquisição da Massima Revestimentos...,6110.0
63,1.02.04.02.10,Ativo Total,Ativo Não Circulante,Intangível,Goodwill,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


### <h4>Drop rows</h4>

Select rows with 4 elements:

In [54]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA_4,DS_CONTA,VL_CONTA
0,1.01.01.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.01.02.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,1.01.02.03,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas ao Custo Amor...,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
6,1.01.03.01,Ativo Total,Ativo Circulante,Contas a Receber,Clientes,Clientes,1239315.0
9,1.01.03.02,Ativo Total,Ativo Circulante,Contas a Receber,Outras Contas a Receber,Outras Contas a Receber,79428.0
11,1.01.04.00,Ativo Total,Ativo Circulante,Estoques,Estoques,Estoques,924743.0
12,1.01.05.00,Ativo Total,Ativo Circulante,Ativos Biológicos,Ativos Biológicos,Ativos Biológicos,0.0
13,1.01.06.01,Ativo Total,Ativo Circulante,Tributos a Recuperar,Tributos Correntes a Recuperar,Tributos Correntes a Recuperar,176456.0
14,1.01.07.00,Ativo Total,Ativo Circulante,Despesas Antecipadas,Despesas Antecipadas,Despesas Antecipadas,0.0


For each key which was used to join, check if there is at least another row in `CD_CONTA` which code starts with the same key (example: `CD_CONTA` '1.01' starting with '1'; `CD_CONTA` '1.01.01' starting with '1.01', etc.)

In [55]:
for idx in idx_list:
    mask = [True]*len(ds_conta_4)
    mask[idx] = False
    if any(row[:round] == cd_conta_joincol[idx] for row in cd_conta_split[mask]):
        df = df.drop(idx)
    else:
        df.loc[idx, 'CD_CONTA'] = df.loc[idx, 'CD_CONTA'] + '.00'

df.reset_index(inplace=True, drop=True)

In [56]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA_4,DS_CONTA,VL_CONTA
0,1.01.01.00.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos para Negociação,0.0
2,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
3,1.01.02.02.00,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.01.02.03.00,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas ao Custo Amor...,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
5,1.01.03.01.01,Ativo Total,Ativo Circulante,Contas a Receber,Clientes,Contas a receber de clientes,1229995.0
6,1.01.03.01.02,Ativo Total,Ativo Circulante,Contas a Receber,Clientes,Contas a receber de partes relacionadas,9320.0
7,1.01.03.02.01,Ativo Total,Ativo Circulante,Contas a Receber,Outras Contas a Receber,Valores a receber,79428.0
8,1.01.04.00.00,Ativo Total,Ativo Circulante,Estoques,Estoques,Estoques,924743.0
9,1.01.05.00.00,Ativo Total,Ativo Circulante,Ativos Biológicos,Ativos Biológicos,Ativos Biológicos,0.0


## <h3>ROUND 5</h3>

In [57]:
round=5

In [58]:
df

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA_4,DS_CONTA,VL_CONTA
0,1.01.01.00.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos para Negociação,0.0
2,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
3,1.01.02.02.00,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.01.02.03.00,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas ao Custo Amor...,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
5,1.01.03.01.01,Ativo Total,Ativo Circulante,Contas a Receber,Clientes,Contas a receber de clientes,1229995.0
6,1.01.03.01.02,Ativo Total,Ativo Circulante,Contas a Receber,Clientes,Contas a receber de partes relacionadas,9320.0
7,1.01.03.02.01,Ativo Total,Ativo Circulante,Contas a Receber,Outras Contas a Receber,Valores a receber,79428.0
8,1.01.04.00.00,Ativo Total,Ativo Circulante,Estoques,Estoques,Estoques,924743.0
9,1.01.05.00.00,Ativo Total,Ativo Circulante,Ativos Biológicos,Ativos Biológicos,Ativos Biológicos,0.0


### <h4>Join</h4>

In [59]:
cd_conta_split = pd.Series([string.split('.') for string in df['CD_CONTA']])
cd_conta_joincol = [row[:round] for row in cd_conta_split]

Select rows with 5 elements:

In [60]:
cd_conta_len = [len(lst) for lst in cd_conta_split]
idx_list = [idx for idx, element in enumerate(cd_conta_len) if element == round]
keys = [cd_conta_joincol[idx] for idx in idx_list]
key_idx_dict = {".".join(key): idx for key, idx in zip(keys, idx_list)}

In [61]:
df.iloc[idx_list]

,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA_4,DS_CONTA,VL_CONTA
0,1.01.01.00.00,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,1728413.0
1,1.01.02.01.01,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos para Negociação,0.0
2,1.01.02.01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Títulos Designados a Valor Justo,0.0
3,1.01.02.02.00,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas a Valor Justo...,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
4,1.01.02.03.00,Ativo Total,Ativo Circulante,Aplicações Financeiras,Aplicações Financeiras Avaliadas ao Custo Amor...,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0
5,1.01.03.01.01,Ativo Total,Ativo Circulante,Contas a Receber,Clientes,Contas a receber de clientes,1229995.0
6,1.01.03.01.02,Ativo Total,Ativo Circulante,Contas a Receber,Clientes,Contas a receber de partes relacionadas,9320.0
7,1.01.03.02.01,Ativo Total,Ativo Circulante,Contas a Receber,Outras Contas a Receber,Valores a receber,79428.0
8,1.01.04.00.00,Ativo Total,Ativo Circulante,Estoques,Estoques,Estoques,924743.0
9,1.01.05.00.00,Ativo Total,Ativo Circulante,Ativos Biológicos,Ativos Biológicos,Ativos Biológicos,0.0


Join:

In [62]:
ds_conta_5 = [df.DS_CONTA[key_idx_dict[key]]
              for key in key_idx_dict
              for row in cd_conta_joincol
              if ".".join(row) == key]

In [63]:
ds_conta_5 = [df.DS_CONTA[key_idx_dict[key]]
              for row in cd_conta_split
              for key in key_idx_dict
              if ".".join(row[:round]) == key]

In [64]:
if pd.Series(ds_conta_5).equals(df.DS_CONTA):
    df.rename(columns={'DS_CONTA': 'DS_CONTA_5'}, inplace=True)
    df.to_csv("output.csv", index=False)
    sys.exit()
else:
    df.insert(round, column='DS_CONTA_5', value=ds_conta_5)

SystemExit: 

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
